In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("../")

import torch
import transformers
import baukit
from tqdm.auto import tqdm
import json
import os
from src import functional
import src.tokens as tokenization_utils

torch.__version__, transformers.__version__, torch.version.cuda

('2.1.2+cu121', '4.36.2', '12.1')

In [3]:
from src.models import ModelandTokenizer

MODEL_PATH = "state-spaces/mamba-2.8b-slimpj" # state-spaces/mamba-2.8b


mt = ModelandTokenizer(
    model_path=MODEL_PATH, 
    torch_dtype=torch.float32
)

/home/local_arnab/miniconda3/envs/relations/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
subject = "The Space Needle"
prompt_template = "{} is located in the city of"

prompt = tokenization_utils.maybe_prefix_eos(mt, prompt_template.format(subject))

functional.predict_next_token(
    mt = mt, 
    prompt = prompt,
)

>>> input_ids.shape= torch.Size([1, 11]) <<<
delta.shape=torch.Size([1, 11, 5120]), A.shape=torch.Size([5120, 16]), B.shape=torch.Size([1, 11, 16]), C.shape=torch.Size([1, 11, 16]), D.shape=torch.Size([5120])
delta.shape=torch.Size([1, 11, 5120]), A.shape=torch.Size([5120, 16]), B.shape=torch.Size([1, 11, 16]), C.shape=torch.Size([1, 11, 16]), D.shape=torch.Size([5120])
delta.shape=torch.Size([1, 11, 5120]), A.shape=torch.Size([5120, 16]), B.shape=torch.Size([1, 11, 16]), C.shape=torch.Size([1, 11, 16]), D.shape=torch.Size([5120])
delta.shape=torch.Size([1, 11, 5120]), A.shape=torch.Size([5120, 16]), B.shape=torch.Size([1, 11, 16]), C.shape=torch.Size([1, 11, 16]), D.shape=torch.Size([5120])
delta.shape=torch.Size([1, 11, 5120]), A.shape=torch.Size([5120, 16]), B.shape=torch.Size([1, 11, 16]), C.shape=torch.Size([1, 11, 16]), D.shape=torch.Size([5120])
delta.shape=torch.Size([1, 11, 5120]), A.shape=torch.Size([5120, 16]), B.shape=torch.Size([1, 11, 16]), C.shape=torch.Size([1, 11, 16])

[[PredictedToken(token=' Seattle', prob=0.9798887372016907),
  PredictedToken(token=' Se', prob=0.0017078507225960493),
  PredictedToken(token=' the', prob=0.0015009533381089568),
  PredictedToken(token=' Sea', prob=0.0008902765694074333),
  PredictedToken(token=' se', prob=0.0006061139283701777)]]

In [127]:
from src.hooking.mamba import selective_scan_with_mask

tokenized = mt.tokenizer(prompt, return_tensors="pt", padding=True, return_offsets_mapping=True).to(mt.device)
offsets = tokenized.pop("offset_mapping")

subj_first, subj_last = functional.find_token_range(
    string = prompt,
    substring=subject,
    offset_mapping=offsets[0],
)

subj_last -= 1
last_idx = tokenized.input_ids.shape[1] - 1

In [177]:
import types

mamba_block_format = "layers.{}.mixer"

def mask_layers(model, layer_indices, mask_config):
    masked_scan = lambda self, u, delta, A, B, C, D : selective_scan_with_mask(
        self = self, u = u, delta = delta, A = A, B = B, C = C, D = D, mask = mask_config
    )
    for layer_idx in layer_indices:
        block = baukit.get_module(model, mamba_block_format.format(layer_idx))
        block.selective_scan = types.MethodType(masked_scan, block)

mt.reset_forward()
mask_everything = {
    idx : torch.arange(0, idx + 1).tolist()
    for idx in range(last_idx + 1)
}

mask_layers(
    model = mt.model,
    # layer_indices = [20],
    layer_indices = torch.arange(0, mt.n_layer).tolist(),
    # mask_config = {last_idx: []}
    mask_config = {last_idx: [subj_last]}
    # mask_config = {last_idx: torch.arange(subj_first, subj_last + 1).tolist()}
    # mask_config = mask_everything
)

In [178]:
# mt.reset_forward()

functional.predict_next_token(
    mt = mt, 
    prompt = prompt,
)

>>> input_ids.shape= torch.Size([1, 11]) <<<
subtracting src_idx=4 from target_idx=10 >> 0.03283882141113281
y.norm()=tensor(3.9454, device='cuda:0')
y[:, target_idx].norm()=tensor(2.2531, device='cuda:0') | IS IT ZERO: False | y[:, target_idx].max()=tensor(0.6197, device='cuda:0') | y[:, target_idx].min()=tensor(-0.9922, device='cuda:0')
y[:, target_idx].norm()=tensor(0.2861, device='cuda:0') | IS IT ZERO: False | y[:, target_idx].max()=tensor(0.0965, device='cuda:0') | y[:, target_idx].min()=tensor(-0.0823, device='cuda:0')
y[:, target_idx].norm()=tensor(0.8665, device='cuda:0') | IS IT ZERO: False | y[:, target_idx].max()=tensor(0.2734, device='cuda:0') | y[:, target_idx].min()=tensor(-0.2156, device='cuda:0')
y[:, target_idx].norm()=tensor(1.9509, device='cuda:0') | IS IT ZERO: False | y[:, target_idx].max()=tensor(0.3006, device='cuda:0') | y[:, target_idx].min()=tensor(-0.4941, device='cuda:0')
y[:, target_idx].norm()=tensor(0.9365, device='cuda:0') | IS IT ZERO: False | y[:, tar

y[:, target_idx].norm()=tensor(2.9622, device='cuda:0') | IS IT ZERO: False | y[:, target_idx].max()=tensor(0.2471, device='cuda:0') | y[:, target_idx].min()=tensor(-0.6250, device='cuda:0')
y[:, target_idx].norm()=tensor(4.0829, device='cuda:0') | IS IT ZERO: False | y[:, target_idx].max()=tensor(0.3617, device='cuda:0') | y[:, target_idx].min()=tensor(-0.8318, device='cuda:0')
y[:, target_idx].norm()=tensor(2.7197, device='cuda:0') | IS IT ZERO: False | y[:, target_idx].max()=tensor(0.2032, device='cuda:0') | y[:, target_idx].min()=tensor(-0.5476, device='cuda:0')
-------------------------------------
subtracting src_idx=4 from target_idx=10 >> 0.10364426672458649
y.norm()=tensor(11.4545, device='cuda:0')
y[:, target_idx].norm()=tensor(1.4967, device='cuda:0') | IS IT ZERO: False | y[:, target_idx].max()=tensor(0.2535, device='cuda:0') | y[:, target_idx].min()=tensor(-0.1744, device='cuda:0')
y[:, target_idx].norm()=tensor(1.6277, device='cuda:0') | IS IT ZERO: False | y[:, target_id

[[PredictedToken(token=' Seattle', prob=0.9759063720703125),
  PredictedToken(token=' Se', prob=0.002054022392258048),
  PredictedToken(token=' the', prob=0.0013701231218874454),
  PredictedToken(token=' Pier', prob=0.0007331949309445918),
  PredictedToken(token=' se', prob=0.0006220503710210323)]]

In [185]:
from src.utils import experiment_utils
experiment_utils.set_seed(123456)

u = torch.randn(1, 4, 5120)
delta = torch.randn(1, 4, 5120)
A = torch.randn(5120, 16)
B = torch.randn(1, 4, 16)
C = torch.randn(1, 4, 16)
D = torch.randn(5120)

output = selective_scan_with_mask(
    self=None,
    u=u,
    delta=delta,
    A=A,
    B=B,
    C=C,
    D=D,
    # mask = {0:[]}
    mask={0: [0], 1: [0, 1], 2: [0, 1, 2], 3: [0, 1, 2, 3]},
)

output.shape

deltaA.shape=torch.Size([1, 4, 5120, 16]) | deltaB_u.shape=torch.Size([1, 4, 5120, 16])
delta_A_src_to_target.shape=torch.Size([1, 5120, 16])
delta_B_src.shape=torch.Size([1, 5120, 16])
subtracting src_idx=0 from target_idx=0 >> 289.8774719238281
delta_A_src_to_target.shape=torch.Size([1, 5120, 16])
delta_B_src.shape=torch.Size([1, 5120, 16])
subtracting src_idx=0 from target_idx=1 >> 3233.42333984375
delta_A_src_to_target.shape=torch.Size([1, 5120, 16])
delta_B_src.shape=torch.Size([1, 5120, 16])
subtracting src_idx=1 from target_idx=1 >> 182.55360412597656
delta_A_src_to_target.shape=torch.Size([1, 5120, 16])
delta_B_src.shape=torch.Size([1, 5120, 16])
subtracting src_idx=0 from target_idx=2 >> 127030.8671875
delta_A_src_to_target.shape=torch.Size([1, 5120, 16])
delta_B_src.shape=torch.Size([1, 5120, 16])
subtracting src_idx=1 from target_idx=2 >> 23893.58203125
delta_A_src_to_target.shape=torch.Size([1, 5120, 16])
delta_B_src.shape=torch.Size([1, 5120, 16])
subtracting src_idx=2 fro

torch.Size([1, 4, 5120])

In [202]:
delta_A = torch.randn(1, 4, 5120, 16)
src_idx = 2
target_idx = 2

delta_A_src_to_target = torch.prod(delta_A[:, src_idx + 1 : target_idx + 1], dim=1)
delta_A_src_to_target.shape

torch.Size([1, 5120, 16])

In [203]:
delta_B_src = torch.randn(1, 5120, 16)

In [204]:
delta_AB_src = delta_A_src_to_target * delta_B_src

torch.allclose(delta_B_src, delta_AB_src)

True

In [5]:
mt.model

Mamba(
  (embedding): Embedding(50280, 2560)
  (layers): ModuleList(
    (0-63): 64 x ResidualBlock(
      (mixer): MambaBlock(
        (in_proj): Linear(in_features=2560, out_features=10240, bias=False)
        (conv1d): Conv1d(5120, 5120, kernel_size=(4,), stride=(1,), padding=(3,), groups=5120)
        (x_proj): Linear(in_features=5120, out_features=192, bias=False)
        (dt_proj): Linear(in_features=160, out_features=5120, bias=True)
        (out_proj): Linear(in_features=5120, out_features=2560, bias=False)
      )
      (norm): RMSNorm()
    )
  )
  (norm_f): RMSNorm()
  (lm_head): Linear(in_features=2560, out_features=50280, bias=False)
)

In [15]:
block = baukit.get_module(
    mt.model,
    "layers.10.mixer",
)

In [17]:
block.A_log.shape

torch.Size([5120, 16])